# NGIML Inference (Local Runtime)
Local-only notebook: uses local `prepared/` dataset and local checkpoints (no Colab/Drive/Hugging Face).

In [ ]:
from pathlib import Path
import sys

REPO_ROOT = Path.cwd().resolve()
if not (REPO_ROOT / 'src').exists() and (REPO_ROOT.parent / 'src').exists():
    REPO_ROOT = REPO_ROOT.parent
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

PREPARED_ROOT = REPO_ROOT / 'prepared'
RUNS_ROOT = REPO_ROOT / 'runs'

print('Repo root:', REPO_ROOT)
print('Prepared root:', PREPARED_ROOT)
print('Runs root:', RUNS_ROOT)

In [ ]:
from tools.local_infer_helpers import (
    ensure_local_manifest,
    find_latest_checkpoint,
    load_model_from_checkpoint,
)

MANIFEST_PATH = ensure_local_manifest(PREPARED_ROOT)
CKPT_PATH = find_latest_checkpoint(RUNS_ROOT)
model, device, ckpt_info = load_model_from_checkpoint(CKPT_PATH)

print('Manifest:', MANIFEST_PATH)
print('Checkpoint:', CKPT_PATH)
print('Device:', device)
print('Checkpoint info:', ckpt_info)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tools.local_infer_helpers import (
    select_manifest_sample,
    load_image_mask_from_record,
    predict_probability_map,
)

sample = select_manifest_sample(MANIFEST_PATH, split_priority=('test', 'val', 'train'), fake_only=True)
image, gt_mask = load_image_mask_from_record(sample)
pred_prob = predict_probability_map(model, image, device)

img_np = image.permute(1, 2, 0).cpu().numpy()
gt_np = gt_mask[0].cpu().numpy()
pred_np = pred_prob.numpy()
pred_bin = (pred_np >= 0.5).astype(np.float32)

overlay = img_np.copy()
overlay[..., 0] = np.clip(overlay[..., 0] + 0.5 * pred_bin, 0, 1)
overlay[..., 1] = np.clip(overlay[..., 1] * (1.0 - 0.35 * pred_bin), 0, 1)
overlay[..., 2] = np.clip(overlay[..., 2] * (1.0 - 0.35 * pred_bin), 0, 1)

plt.figure(figsize=(14, 4))
plt.subplot(1, 3, 1)
plt.title(f'Photo ({sample.dataset}/{sample.split})')
plt.imshow(img_np)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Prediction')
plt.imshow(pred_np, cmap='magma', vmin=0, vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Overlay')
plt.imshow(overlay)
plt.axis('off')

plt.tight_layout()
plt.show()

print('Prediction stats min/mean/max:', float(pred_np.min()), float(pred_np.mean()), float(pred_np.max()))

In [ ]:
# Optional: run on a custom local image path
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from tools.local_infer_helpers import infer_from_image_path

CUSTOM_IMAGE_PATH = None  # e.g., REPO_ROOT / 'my_image.jpg'
if CUSTOM_IMAGE_PATH:
    img, pred = infer_from_image_path(model, Path(CUSTOM_IMAGE_PATH), device)
    img_np = img.permute(1, 2, 0).cpu().numpy()
    pred_np = pred.numpy()
    pred_bin = (pred_np >= 0.5).astype(np.float32)

    overlay = img_np.copy()
    overlay[..., 0] = np.clip(overlay[..., 0] + 0.5 * pred_bin, 0, 1)
    overlay[..., 1] = np.clip(overlay[..., 1] * (1.0 - 0.35 * pred_bin), 0, 1)
    overlay[..., 2] = np.clip(overlay[..., 2] * (1.0 - 0.35 * pred_bin), 0, 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.title('Custom Photo')
    plt.imshow(img_np)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title('Prediction')
    plt.imshow(pred_np, cmap='magma', vmin=0, vmax=1)
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title('Overlay')
    plt.imshow(overlay)
    plt.axis('off')

    plt.tight_layout()
    plt.show()
else:
    print('Set CUSTOM_IMAGE_PATH to run custom-image inference.')